2-274-03-00788 processing progress: 0.13%
2-274-03-00788 processing progress: 0.25%
2-274-03-00788 processing progress: 0.38%
2-274-03-00788 processing progress: 0.51%
2-274-03-00788 processing progress: 0.64%
2-274-03-00788 processing progress: 0.76%
2-274-03-00788 processing progress: 0.89%
2-274-03-00788 processing progress: 1.02%
2-274-03-00788 processing progress: 1.14%
2-274-03-00788 processing progress: 1.27%
2-274-03-00788 processing progress: 1.40%
2-274-03-00788 processing progress: 1.52%
2-274-03-00788 processing progress: 1.65%
2-274-03-00788 processing progress: 1.78%
2-274-03-00788 processing progress: 1.91%
2-274-03-00788 processing progress: 2.03%
2-274-03-00788 processing progress: 2.16%
2-274-03-00788 processing progress: 2.29%
2-274-03-00788 processing progress: 2.41%
2-274-03-00788 processing progress: 2.54%
2-274-03-00788 processing progress: 2.67%
2-274-03-00788 processing progress: 2.80%
2-274-03-00788 processing progress: 2.92%
2-274-03-00788 processing progress

In [1]:
import os
import cv2
from multiprocessing import Process, Queue, Lock, cpu_count, Event, Value, Manager, cpu_count
from resources.utils import process_image
import threading
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor, as_completed

def process_and_save_image(filename, images_path, temp_images_path, zoom_percentage, counter, lock, print):
    if not filename.endswith('.tif') or 'thumb' in filename:
        return
    image = cv2.imread(os.path.join(images_path, filename))
    halfs = process_image(image, zoom_percentage)
    labels = ['left', 'right']
    for half, label in zip(halfs, labels):
        cv2.imwrite(os.path.join(temp_images_path, f"{filename.split('.')[0]}_{label}.png"), half)

def process_images(images_path, temp_images_path, zoom_percentage, cache=False, print=print):
    # Remove existing files in temp_images_path
    for filename in os.listdir(temp_images_path):
        os.remove(os.path.join(temp_images_path, filename))

    # Create a manager for shared objects
    manager = Manager()
    counter = manager.Value('i', 0)
    lock = manager.Lock()
    total_files = len([f for f in os.listdir(images_path) if f.endswith('.tif') and 'thumb' not in f])
    counter.total = total_files

    filenames = os.listdir(images_path)

    # Use ProcessPoolExecutor for parallel processing
    with ProcessPoolExecutor(max_workers=min(4, cpu_count())) as executor:
        futures = [executor.submit(process_and_save_image, filename, images_path, temp_images_path, zoom_percentage, counter, lock, print) for filename in filenames]
        for future in as_completed(futures):
            future.result()  # Ensure exceptions are raised


folder_dir = '/media/michalis/T/2-274-03-00788'
temp_dir = os.getcwd() + '/temp_images/' + os.path.basename(folder_dir)

process_images(folder_dir, temp_dir, 0)